In [1]:
import opal
import kglab
import rdflib
import z3
import dateutil.parser
import re
import numpy as np
import pandas as pd
import os
import sexpdata

In [2]:
from opal.logic.z3.z3_literal import Z3Literal
from opal.data.event_data import EventData
from opal.data.mapper import LogMapper
from opal.data.processor import LogProcessor

import opal.logic.z3.z3_literal as z3_literal
from opal.logic.z3.z3_ontology import Z3Ontology as Ontology
from opal.logic.z3.z3_reasoner import Z3Reasoner as Reasoner
from opal.logic.z3.z3_ontology import REF_ONTOLOGY_Z3, PSL_CORE_Z3, PSL_COMPLEX_Z3, PSL_OCCTREE_Z3, PSL_SUBACTIVITY_Z3, PSL_subactivity_occtree, PSL_ATOMIC_Z3

from rdflib import Graph

from z3 import *
z3.set_param(proof=True)
z3.set_param('verbose', 0)

Adding axiom: Reference Taxonomy_ont_t_0_disjoint_classes
Adding axiom: Reference Taxonomy_ont_t_1_event_order_SIMPLE
Adding axiom: Reference Taxonomy_ont_t_2_infer_next_event
Adding axiom: PSL Core Ontology_ont_t_0_type_disjointness
Adding axiom: PSL Core Ontology_ont_t_1_begin_unique
Adding axiom: PSL Core Ontology_ont_t_2_ends_unique
Adding axiom: PSL Core Ontology_ont_t_3_occurrence_bounds
Adding axiom: PSL Core Ontology_ont_t_4_occurrence_sort_constraints
Adding axiom: PSL Core Ontology_ont_t_5_unique_activity_occurrence
Adding axiom: PSL Core Ontology_ont_t_6_occurrence_has_activity
Adding axiom: PSL Core Ontology_ont_t_7_participation_t_sort_constraints
Adding axiom: PSL Core Ontology_ont_t_8_participation_time_to_generic
Adding axiom: PSL Core Ontology_ont_t_9_participation_sort_constraints
Adding axiom: PSL Core Ontology_ont_t_10_participation_temporal_constraint
Adding axiom: PSL Core Ontology_ont_t_11_object_temporal_existence
Adding axiom: PSL Core Ontology_ont_t_12_occurre

In [3]:
data_dir = os.path.abspath('../data/')
ttl_data_file = data_dir + '/P1_log_instances.ttl'
sample_log_file = data_dir + '/sample_log.csv'
bpic2017_file = data_dir + '/BPI Challenge 2017.xes.gz'

ontology_dir = os.path.abspath('../../opal/logic/z3/ontologies/')
psl_core_file = ontology_dir + '/PSL/PSL_core.smt2'
psl_core_mapping_file = ontology_dir + '/PSL/PSL_core_mapping.smt2'

## Mapping and reasoning on a simple XES log

### Mapping

In [4]:
# load the log
col_dict = {'caseID' : 'caseID', 'activityID' : 'activityID', 'timestamp' : 'timestamp', 'eventID' : None, 'transition' : 'transition'}
processor = LogProcessor(col_dict=col_dict, log_path=sample_log_file, batch_size=1)
event_data = processor.load_log()

In [5]:
# viewing the original data
event_data.data

caseID    activityID eventID                 timestamp resourceID  \
0  C_case_0  A_activity_A     E_0 2016-01-01 09:00:00+00:00     user_1   
1  C_case_0  A_activity_A     E_1 2016-01-01 09:15:00+00:00     user_1   
2  C_case_0  A_activity_C     E_2 2016-01-01 09:35:00+00:00     user_1   
3  C_case_1  A_activity_A     E_3 2016-01-02 09:00:00+00:00     user_1   
4  C_case_1  A_activity_B     E_4 2016-01-02 09:00:00+00:00     user_0   
5  C_case_1  A_activity_C     E_5 2016-01-03 09:00:00+00:00     user_0   
6  C_case_2  A_activity_A     E_6 2016-01-01 09:00:00+00:00     user_1   
7  C_case_2  A_activity_B     E_7 2016-01-01 09:10:00+00:00     user_1   
8  C_case_2  A_activity_C     E_8 2016-01-01 09:15:00+00:00     user_2   
9  C_case_2  A_activity_D     E_9 2017-01-06 10:35:00+00:00     user_1   

   transition processID  batch  
0     T_start        P1      1  
1  T_complete        P1      1  
2  T_complete        P1      1  
3  T_complete        P1      2  
4  T_complete        P1      2  
5  T_complete        P1      2  
6  T_complete        P1      3  
7  T_complete        P1      3  
8  T_complete        P1      3  
9  T_complete        P1      3

In [6]:
# metadata is automatically created
event_data.metadata

{'log_paths': ['/tmp/tmpiubdh_l_.csv',
  '/tmp/tmpej1x9s_9.csv',
  '/tmp/tmpfscag72b.csv'],
 'process_name': 'P1',
 'batches': 3}

In [7]:
# initialize a mapper to map the log to logical facts according to the specified encoding (here, z3)
mapper = LogMapper(data=event_data, output_encoding='z3')
z3_data = mapper.get_mapped_data()

Building mappings for logs...


  0%|          | 0/3 [00:00<?, ?it/s]2025-10-22 17:29:54,125 | INFO: Translating YARRRML mapping to [R2]RML
2025-10-22 17:29:54,126 | INFO: RML content is created!
2025-10-22 17:29:54,133 | INFO: Mapping has been syntactically validated.
2025-10-22 17:29:54,133 | INFO: Translation has finished successfully.
2025-10-22 17:29:54,140 | INFO: Translating YARRRML mapping to [R2]RML
2025-10-22 17:29:54,141 | INFO: RML content is created!
2025-10-22 17:29:54,147 | INFO: Mapping has been syntactically validated.
2025-10-22 17:29:54,148 | INFO: Translation has finished successfully.
2025-10-22 17:29:54,156 | INFO: Translating YARRRML mapping to [R2]RML
2025-10-22 17:29:54,157 | INFO: RML content is created!
2025-10-22 17:29:54,162 | INFO: Mapping has been syntactically validated.
2025-10-22 17:29:54,162 | INFO: Translation has finished successfully.
100%|██████████| 3/3 [00:00<00:00, 67.19it/s]

Mappings built.
Generating knowledge graph...



INFO | 2025-10-22 17:29:54,599 | 11 mapping rules retrieved.
INFO | 2025-10-22 17:29:54,609 | Mapping partition with 11 groups generated.
INFO | 2025-10-22 17:29:54,610 | Maximum number of rules within mapping group: 1.
INFO | 2025-10-22 17:29:54,610 | Mappings processed in 0.442 seconds.
INFO | 2025-10-22 17:29:55,003 | Number of triples generated in total: 25.


Knowledge graph generated.
Knowledge graph 1/3 generated.
Generating knowledge graph...


INFO | 2025-10-22 17:29:55,428 | 11 mapping rules retrieved.
INFO | 2025-10-22 17:29:55,438 | Mapping partition with 11 groups generated.
INFO | 2025-10-22 17:29:55,438 | Maximum number of rules within mapping group: 1.
INFO | 2025-10-22 17:29:55,439 | Mappings processed in 0.417 seconds.
INFO | 2025-10-22 17:29:55,811 | Number of triples generated in total: 26.


Knowledge graph generated.
Knowledge graph 2/3 generated.
Generating knowledge graph...


INFO | 2025-10-22 17:29:56,214 | 11 mapping rules retrieved.
INFO | 2025-10-22 17:29:56,223 | Mapping partition with 11 groups generated.
INFO | 2025-10-22 17:29:56,224 | Maximum number of rules within mapping group: 1.
INFO | 2025-10-22 17:29:56,224 | Mappings processed in 0.406 seconds.
INFO | 2025-10-22 17:29:56,586 | Number of triples generated in total: 33.


Knowledge graph generated.
Knowledge graph 3/3 generated.
Z3 literal set (1/3) generated.
Z3 literal set (2/3) generated.
Z3 literal set (3/3) generated.


In [8]:
# rdf is still generated, because it is a prerequisite for mapping to more expressive encoding languages
graph = mapper.kgs[0].rdf_graph()
for triple in graph.triples((None, None, None)):
    print(triple)

(rdflib.term.URIRef('http://www.example.com/E_1'), rdflib.term.URIRef('https://stl.mie.utoronto.ca/ontologies/spm/hasTransition'), rdflib.term.URIRef('http://www.example.com/T_complete'))
(rdflib.term.URIRef('http://www.example.com/E_0'), rdflib.term.URIRef('https://stl.mie.utoronto.ca/ontologies/spm/hasCase'), rdflib.term.URIRef('http://www.example.com/C_case_0'))
(rdflib.term.URIRef('http://www.example.com/E_1'), rdflib.term.URIRef('https://stl.mie.utoronto.ca/ontologies/spm/hasActivity'), rdflib.term.URIRef('http://www.example.com/A_activity_A'))
(rdflib.term.URIRef('http://www.example.com/E_0'), rdflib.term.URIRef('https://stl.mie.utoronto.ca/ontologies/spm/hasResource'), rdflib.term.URIRef('http://www.example.com/user_1'))
(rdflib.term.URIRef('http://www.example.com/user_1'), rdflib.term.URIRef('http://www.w3.org/1999/02/22-rdf-syntax-ns#type'), rdflib.term.URIRef('https://stl.mie.utoronto.ca/ontologies/spm/Resource'))
(rdflib.term.URIRef('http://www.example.com/E_0'), rdflib.term

In [ ]:
from opal.logic.z3.z3_handler import PSL_CORE_MAPPING_ENV
for literal in z3_data:
    expr = literal.z3_expr
    print(expr)
    
# get a random expression to check if contexts remain the same with additional ontologies loaded
random_idx = np.random.randint(len(z3_data))
random_expr = z3_data[random_idx].z3_expr
print(f'Matching contexts: {random_expr.ctx == PSL_CORE_MAPPING_ENV["ctx"]}')

In [ ]:
PSL_CORE_MAPPING_ENV['functions'] 

{'Event': Event,
 'Transition': Transition,
 'Activity': Activity,
 'Case': Case,
 'Resource': Resource,
 'hasResource': hasResource,
 'hasRecordedTime': hasRecordedTime,
 'hasLifecycleTransition': hasLifecycleTransition,
 'hasCase': hasCase,
 'hasActivity': hasActivity,
 'next_event': next_event,
 'event_earlier': event_earlier,
 'activity': activity,
 'activity_occurrence': activity_occurrence,
 'object_': object_,
 'timepoint': timepoint,
 'occurrence_of': occurrence_of,
 'participates_in': participates_in,
 'participates': participates,
 'exists_at': exists_at,
 'is_occurring_at': is_occurring_at,
 'begin_of': begin_of,
 'end_of': end_of}

### Reasoning Capabilities

In [ ]:
# load the ontology (together with the mapping)
psl_core = Ontology.from_smt2(psl_core_file, psl_core_mapping_file)

Adding axiom: None_ont_t_0_type_disjointness
Adding axiom: None_ont_t_1_begin_unique
Adding axiom: None_ont_t_2_ends_unique
Adding axiom: None_ont_t_3_occurrence_bounds
Adding axiom: None_ont_t_4_occurrence_sort_constraints
Adding axiom: None_ont_t_5_unique_activity_occurrence
Adding axiom: None_ont_t_6_occurrence_has_activity
Adding axiom: None_ont_t_7_participation_t_sort_constraints
Adding axiom: None_ont_t_8_participation_time_to_generic
Adding axiom: None_ont_t_9_participation_sort_constraints
Adding axiom: None_ont_t_10_participation_temporal_constraint
Adding axiom: None_ont_t_11_object_temporal_existence
Adding axiom: None_ont_t_12_occurrence_temporal_extent
Adding axiom: None_map_t_0_occurrence_start_end_event_mapping
Adding axiom: None_map_t_1_occurrence_participation_mapping


In [ ]:
psl_core.axioms

[{'name': 'None_ont_t_0_type_disjointness',
  'description': 'Activities, occurrences, and objects are different things.',
  'expr': '(assert (! (forall ((x Ind)) (and (=> (activity x) (not (or (activity_occurrence x) (object_ x)))) (=> (activity_occurrence x) (not (or (object_ x) (activity x)))) (=> (object_ x) (not (or (activity_occurrence x) (activity x)))))) :named type_disjointness :description "Activities, occurrences, and objects are different things."))'},
 {'name': 'None_ont_t_1_begin_unique',
  'description': 'Start points are unique.',
  'expr': '(assert (! (forall ((x Ind) (t1 Real) (t2 Real)) (=> (and (= (begin_of x) t1) (= (begin_of x) t2)) (= t1 t2))) :named begin_unique :description "Start points are unique."))'},
 {'name': 'None_ont_t_2_ends_unique',
  'description': 'End points are unique.',
  'expr': '(assert (! (forall ((x Ind) (t1 Real) (t2 Real)) (=> (and (= (end_of x) t1) (= (end_of x) t2)) (= t1 t2))) :named ends_unique :description "End points are unique."))'},

In [ ]:
z3_facts = []
for i, fact in enumerate(z3_data):
    prefix = f'mapped_a_{i}_'
    expr = fact
    name = prefix + f'{expr}'
    z3_facts.append({name : expr})

In [ ]:
z3_facts[:5]

[{'mapped_a_0_hasRecordedTime(E_2) == 1451640900': <opal.logic.z3.z3_literal.Z3Literal at 0x7f3b64192a60>},
 {'mapped_a_1_Resource(user_1)': <opal.logic.z3.z3_literal.Z3Literal at 0x7f3b6418dac0>},
 {'mapped_a_2_hasTransition(E_0, T_start)': <opal.logic.z3.z3_literal.Z3Literal at 0x7f3b6418db80>},
 {'mapped_a_3_hasActivity(E_0, A_activity_A)': <opal.logic.z3.z3_literal.Z3Literal at 0x7f3b6418dbb0>},
 {'mapped_a_4_hasCase(E_1, C_case_0)': <opal.logic.z3.z3_literal.Z3Literal at 0x7f3b6418d850>}]

In [ ]:
module = PSL_COMPLEX_Z3
r = Reasoner()
r.load_ontology(module)
r.load_data_declarations(mapper._z3_env)
r.load_facts(z3_facts)
r.add_relation_closure_axiom('hasCase')
r.add_relation_closure_axiom('Event')
r.add_relation_closure_axiom('Case')
r.add_unique_name_axioms('Event')
r.add_unique_name_axioms('Case')
s = r.check()
stats = s.statistics()
s.reset()
mem = stats.get_key_value('max memory')
time = stats.get_key_value('time')
print(f'Module: {module.name}, Memory: {mem} MB, Time: {time} s')

Loading assertion: Reference Taxonomy_ont_t_0_disjoint_classes : (assert (! (forall ((x Ind)) (and (=> (Event x) (not (or (Transition x) (Activity x) (Case x) (Resource x)))) (=> (Transition x) (not (or (Activity x) (Case x) (Resource x)))) (=> (Activity x) (not (or (Case x) (Resource x)))) (=> (Case x) (not (Resource x))))) :named disjoint_classes :description "Events, transitions, activities, cases, and resources are disjoint."))
Loading assertion: Reference Taxonomy_ont_t_1_event_order_SIMPLE : (assert (! (forall ((e1 Ind) (e2 Ind) (c Ind)) (= (and (Event e1) (Event e2) (Case c) (hasCase e1 c) (hasCase e2 c) (< (hasRecordedTime e1) (hasRecordedTime e2))) (event_earlier e1 e2 c))) :named event_order_SIMPLE))
Loading assertion: Reference Taxonomy_ont_t_2_infer_next_event : (assert (! (forall ((e1 Ind) (e2 Ind) (c Ind)) (= (and (event_earlier e1 e2 c) (not (exists ((e3 Ind) (t3 Real)) (and (Event e3) (not (= e3 e1)) (not (= e3 e2)) (hasCase e3 c) (< (hasRecordedTime e3) (hasRecordedTim

Module: PSL Atomic Ontology, Memory: 633.92 MB, Time: 5.162 s


In [ ]:
stats_dict = {}
for module in [PSL_CORE_Z3, PSL_OCCTREE_Z3, PSL_SUBACTIVITY_Z3, PSL_ATOMIC_Z3, PSL_COMPLEX_Z3]:
    # init a reasoner and load facts
    r = Reasoner()
    r.load_ontology(module)
    r.load_data_declarations(mapper._z3_env)
    r.load_facts(z3_facts)
    r.add_relation_closure_axiom('hasCase')
    r.add_relation_closure_axiom('Event')
    r.add_relation_closure_axiom('Case')
    r.add_unique_name_axioms('Event')
    r.add_unique_name_axioms('Case')
    print(f'Loaded module: {module.name}')
    s = r.check()
    print(f'Reasoner ran with module {module.name}')
    stats = s.statistics()
    s.reset()
    mem = stats.get_key_value('max memory')
    time = stats.get_key_value('time')
    print(f'Module: {module.name}, Memory: {mem} MB, Time: {time} s')
    stats_dict[module.name] = {'memory_MB' : mem, 'time_s' : time}
pretty_stats = pd.DataFrame.from_dict(stats_dict, orient='index')
print(pretty_stats)

Loading assertion: Reference Taxonomy_ont_t_0_disjoint_classes : (assert (! (forall ((x Ind)) (and (=> (Event x) (not (or (Transition x) (Activity x) (Case x) (Resource x)))) (=> (Transition x) (not (or (Activity x) (Case x) (Resource x)))) (=> (Activity x) (not (or (Case x) (Resource x)))) (=> (Case x) (not (Resource x))))) :named disjoint_classes :description "Events, transitions, activities, cases, and resources are disjoint."))
Loading assertion: Reference Taxonomy_ont_t_1_event_order_SIMPLE : (assert (! (forall ((e1 Ind) (e2 Ind) (c Ind)) (= (and (Event e1) (Event e2) (Case c) (hasCase e1 c) (hasCase e2 c) (< (hasRecordedTime e1) (hasRecordedTime e2))) (event_earlier e1 e2 c))) :named event_order_SIMPLE))
Loading assertion: Reference Taxonomy_ont_t_2_infer_next_event : (assert (! (forall ((e1 Ind) (e2 Ind) (c Ind)) (= (and (event_earlier e1 e2 c) (not (exists ((e3 Ind) (t3 Real)) (and (Event e3) (not (= e3 e1)) (not (= e3 e2)) (hasCase e3 c) (< (hasRecordedTime e3) (hasRecordedTim

Reasoner ran with module PSL Core Ontology
Module: PSL Core Ontology, Memory: 565.46 MB, Time: 17.068 s
Loading assertion: Reference Taxonomy_ont_t_0_disjoint_classes : (assert (! (forall ((x Ind)) (and (=> (Event x) (not (or (Transition x) (Activity x) (Case x) (Resource x)))) (=> (Transition x) (not (or (Activity x) (Case x) (Resource x)))) (=> (Activity x) (not (or (Case x) (Resource x)))) (=> (Case x) (not (Resource x))))) :named disjoint_classes :description "Events, transitions, activities, cases, and resources are disjoint."))
Loading assertion: Reference Taxonomy_ont_t_1_event_order_SIMPLE : (assert (! (forall ((e1 Ind) (e2 Ind) (c Ind)) (= (and (Event e1) (Event e2) (Case c) (hasCase e1 c) (hasCase e2 c) (< (hasRecordedTime e1) (hasRecordedTime e2))) (event_earlier e1 e2 c))) :named event_order_SIMPLE))
Loading assertion: Reference Taxonomy_ont_t_2_infer_next_event : (assert (! (forall ((e1 Ind) (e2 Ind) (c Ind)) (= (and (event_earlier e1 e2 c) (not (exists ((e3 Ind) (t3 Real)

In [18]:
pretty_stats

memory_MB  time_s
PSL Core Ontology            193.65   8.438
PSL Complex Ontology         620.53  19.470
PSL Occtree Ontology         197.96   5.352
PSL Subactivity Ontology     338.10  20.814
PSL Atomic Ontology          620.53  30.105

In [ ]:
s = r.check()

In [26]:
# Since the data is consistent with the ontology, we have a satisfying interpretation and can extract a model
print(s.model())

[mapped_a_15_hasResource(E_2, user_1) = True,
 PSL Core Ontology_ont_t_6_occurrence_has_activity = True,
 mapped_a_11_hasActivity(E_2, A_activity_C) = True,
 E_8 = Ind!val!4,
 A_activity_A = Ind!val!16,
 mapped_a_31_Event(E_5) = True,
 mapped_a_42_hasRecordedTime(E_4) == 1451725200 = True,
 E_0 = Ind!val!0,
 Reference Taxonomy_ont_t_2_infer_next_event = True,
 mapped_a_5_Transition(T_start) = True,
 mapped_a_27_Transition(T_complete) = True,
 mapped_a_29_hasTransition(E_5, T_complete) = True,
 mapped_a_36_hasCase(E_5, C_case_1) = True,
 mapped_a_70_Activity(A_activity_C) = True,
 mapped_a_25_Activity(A_activity_C) = True,
 Reference Taxonomy_ont_a_1_transition_complete = True,
 mapped_a_55_Event(E_8) = True,
 Reference Taxonomy_ont_t_1_event_order_SIMPLE = True,
 Reference Taxonomy_ont_a_0_transition_start = True,
 mapped_a_41_hasTransition(E_4, T_complete) = True,
 UniqueNames_Event = True,
 mapped_a_10_hasTransition(E_1, T_complete) = True,
 PSL Core Ontology_ont_t_7_participation_t_

In [19]:
m = r.model
m[r.decls['event_earlier']]

[else ->
 Or(And(k!2734(Var(0)) == Ind!val!0,
        k!2734(Var(1)) == Ind!val!5,
        k!2734(Var(2)) == Ind!val!12),
    And(k!2734(Var(0)) == Ind!val!4,
        k!2734(Var(1)) == Ind!val!9,
        k!2734(Var(2)) == Ind!val!10),
    And(k!2734(Var(0)) == Ind!val!8,
        k!2734(Var(1)) == Ind!val!2,
        k!2734(Var(2)) == Ind!val!10),
    And(k!2734(Var(0)) == Ind!val!2,
        k!2734(Var(1)) == Ind!val!4,
        k!2734(Var(2)) == Ind!val!10),
    And(k!2734(Var(0)) == Ind!val!1,
        k!2734(Var(1)) == Ind!val!6,
        k!2734(Var(2)) == Ind!val!11),
    And(k!2734(Var(0)) == Ind!val!0,
        k!2734(Var(1)) == Ind!val!7,
        k!2734(Var(2)) == Ind!val!12),
    And(k!2734(Var(0)) == Ind!val!3,
        k!2734(Var(1)) == Ind!val!6,
        k!2734(Var(2)) == Ind!val!11),
    And(k!2734(Var(0)) == Ind!val!5,
        k!2734(Var(1)) == Ind!val!7,
        k!2734(Var(2)) == Ind!val!12),
    And(k!2734(Var(0)) == Ind!val!2,
        k!2734(Var(1)) == Ind!val!9,
        k!2734(Var(2)) == Ind!val!10),
    And(k!2734(Var(0)) == Ind!val!8,
        k!2734(Var(1)) == Ind!val!9,
        k!2734(Var(2)) == Ind!val!10),
    And(k!2734(Var(0)) == Ind!val!8,
        k!2734(Var(1)) == Ind!val!4,
        k!2734(Var(2)) == Ind!val!10))]

In [20]:
m[r.decls['next_event']]

[else ->
 Or(And(k!2734(Var(0)) == Ind!val!0,
        k!2734(Var(1)) == Ind!val!5,
        k!2734(Var(2)) == Ind!val!12),
    And(k!2734(Var(0)) == Ind!val!4,
        k!2734(Var(1)) == Ind!val!9,
        k!2734(Var(2)) == Ind!val!10),
    And(k!2734(Var(0)) == Ind!val!8,
        k!2734(Var(1)) == Ind!val!2,
        k!2734(Var(2)) == Ind!val!10),
    And(k!2734(Var(0)) == Ind!val!2,
        k!2734(Var(1)) == Ind!val!4,
        k!2734(Var(2)) == Ind!val!10),
    And(k!2734(Var(0)) == Ind!val!1,
        k!2734(Var(1)) == Ind!val!6,
        k!2734(Var(2)) == Ind!val!11),
    And(k!2734(Var(0)) == Ind!val!3,
        k!2734(Var(1)) == Ind!val!6,
        k!2734(Var(2)) == Ind!val!11),
    And(k!2734(Var(0)) == Ind!val!5,
        k!2734(Var(1)) == Ind!val!7,
        k!2734(Var(2)) == Ind!val!12))]

In [23]:
r.assertions

[('Reference Taxonomy_ont_t_0_disjoint_classes',
  ForAll(x,
         And(Implies(Event(x),
                     Not(Or(Transition(x),
                            Activity(x),
                            Case(x),
                            Resource(x)))),
             Implies(Transition(x),
                     Not(Or(Activity(x), Case(x), Resource(x)))),
             Implies(Activity(x),
                     Not(Or(Case(x), Resource(x)))),
             Implies(Case(x), Not(Resource(x)))))),
 ('Reference Taxonomy_ont_t_1_event_order_SIMPLE',
  ForAll([e1, e2, c],
         And(Event(e1),
             Event(e2),
             Case(c),
             hasCase(e1, c),
             hasCase(e2, c),
             hasRecordedTime(e1) < hasRecordedTime(e2)) ==
         event_earlier(e1, e2, c))),
 ('Reference Taxonomy_ont_t_2_infer_next_event',
  ForAll([e1, e2, c],
         And(event_earlier(e1, e2, c),
             Not(Exists([e3, t3],
                        And(Event(e3),
                      

### Pattern Usage

In [17]:
from opal.logic.z3.z3_pattern import HAND_OFF_PATTERN, PING_PONG_PATTERN, PING_PONG_QUERY_PATTERN
ping_pong_q = PING_PONG_QUERY_PATTERN.apply()
s = r.query(ping_pong_q)

: 

In [22]:
m = s.model()

In [23]:
m

[PSL Core Ontology_ont_t_6_occurrence_has_activity = True,
 E_8 = Ind!val!18,
 A_activity_A = Ind!val!3,
 mapped_a_63_hasResource(E_6, user_1) = True,
 mapped_a_65_Resource(user_1) = True,
 mapped_a_45_hasActivity(E_5, A_activity_C) = True,
 mapped_a_47_hasTransition(E_4, T_complete) = True,
 E_0 = Ind!val!2,
 mapped_a_39_hasRecordedTime(E_4, 1451725200) = True,
 mapped_a_57_Event(E_8) = True,
 mapped_a_82_hasRecordedTime(E_7, 1451639400) = True,
 mapped_a_0_hasTransition(E_2, T_complete) = True,
 mapped_a_23_hasActivity(E_1, A_activity_A) = True,
 mapped_a_64_hasActivity(E_8, A_activity_C) = True,
 mapped_a_29_Activity(A_activity_C) = True,
 mapped_a_59_Transition(T_complete) = True,
 mapped_a_15_hasResource(E_0, user_1) = True,
 E_7 = Ind!val!17,
 PSL Core Ontology_ont_t_7_participation_t_sort_constraints = True,
 mapped_a_66_hasTransition(E_6, T_complete) = True,
 mapped_a_27_hasCase(E_5, C_case_1) = True,
 E_1 = Ind!val!4,
 user_0 = Ind!val!14,
 PSL Core Ontology_ont_t_10_participation_temporal_constraint = True,
 T_complete = Ind!val!1,
 mapped_a_1_hasActivity(E_0, A_activity_A) = True,
 mapped_a_42_hasCase(E_3, C_case_1) = True,
 PSL Core Ontology_ont_t_2_ends_unique = True,
 mapped_a_53_hasCase(E_9, C_case_2) = True,
 user_1 = Ind!val!5,
 mapped_a_60_hasRecordedTime(E_6, 1451638800) = True,
 mapped_a_33_hasTransition(E_3, T_complete) = True,
 mapped_a_67_hasCase(E_7, C_case_2) = True,
 mapped_a_3_Activity(A_activity_C) = True,
 PSL Core Ontology_ont_t_11_object_temporal_existence = True,
 mapped_a_22_hasTransition(E_0, T_start) = True,
 PSL Core Ontology_ont_t_1_begin_unique = True,
 mapped_a_8_hasCase(E_1, C_case_0) = True,
 p_Hand-off_ = True,
 mapped_a_69_Event(E_9) = True,
 PSL Core Ontology_ont_t_0_type_disjointness = True,
 mapped_a_20_hasCase(E_0, C_case_0) = True,
 mapped_a_77_hasActivity(E_6, A_activity_A) = True,
 E_6 = Ind!val!19,
 mapped_a_17_Resource(user_1) = True,
 mapped_a_50_hasActivity(E_3, A_activity_A) = True,
 PSL Core Ontology_map_t_1_occurrence_participation_mapping = True,
 E_2 = Ind!val!0,
 mapped_a_28_Event(E_4) = True,
 PSL Subactivity Ontology_ont_t_1_subactivity_reflexive = True,
 mapped_a_78_Activity(A_activity_A) = True,
 E_9 = Ind!val!16,
 PSL Core Ontology_ont_t_3_occurrence_bounds = True,
 mapped_a_79_hasProcess(C_case_2, P1) = True,
 mapped_a_35_hasTransition(E_5, T_complete) = True,
 PSL Core Ontology_ont_t_9_participation_sort_constraints = True,
 mapped_a_75_Activity(A_activity_C) = True,
 mapped_a_25_hasRecordedTime(E_5, 1451811600) = True,
 mapped_a_43_Resource(user_0) = True,
 mapped_a_5_hasResource(E_2, user_1) = True,
 mapped_a_61_Activity(A_activity_D) = True,
 mapped_a_36_hasResource(E_3, user_1) = True,
 PSL Subactivity Ontology_ont_t_4_subactivity_discrete_up = True,
 A_activity_D = Ind!val!20,
 user_2 = Ind!val!21,
 PSL Subactivity Ontology_ont_t_3_subactivity_transitive = True,
 mapped_a_14_hasActivity(E_2, A_activity_C) = True,
 mapped_a_13_Activity(A_activity_A) = True,
 mapped_a_10_Transition(T_start) = True,
 mapped_a_26_Case(C_case_1) = True,
 mapped_a_49_Event(E_5) = True,
 mapped_a_32_hasProcess(C_case_1, P1) = True,
 mapped_a_80_hasRecordedTime(E_8, 1451639700) = True,
 E_3 = Ind!val!13,
 mapped_a_46_Event(E_3) = True,
 mapped_a_2_hasResource(E_1, user_1) = True,
 mapped_a_71_hasCase(E_6, C_case_2) = True,
 mapped_a_38_Transition(T_complete) = True,
 PSL Core Ontology_ont_t_4_occurrence_sort_constraints = True,
 mapped_a_56_hasCase(E_8, C_case_2) = True,
 mapped_a_62_hasRecordedTime(E_9, 1483698900) = True,
 p_Ping-pong Query_ = True,
 mapped_a_11_Event(E_1) = True,
 PSL Core Ontology_ont_t_12_occurrence_temporal_extent = True,
 mapped_a_40_Resource(user_1) = True,
 mapped_a_44_hasResource(E_5, user_0) = True,
 mapped_a_54_hasResource(E_9, user_1) = True,
 mapped_a_58_hasTransition(E_8, T_complete) = True,
 p_Ping-pong_ = True,
 mapped_a_30_Activity(A_activity_B) = True,
 mapped_a_37_Activity(A_activity_A) = True,
 mapped_a_19_hasRecordedTime(E_1, 1451639700) = Tr

# Mapping BPIC 2017 data

In [12]:
processor = LogProcessor(log_path=bpic2017_file, batch_size=100)
event_data = processor.load_log(downsample_rate=0.05)

/home/vscode/.local/lib/python3.9/site-packages/pm4py/util/dt_parsing/parser.py:82: UserWarning: ISO8601 strings are not fully supported with strpfromiso for Python versions below 3.11
  warnings.warn(
/opt/conda/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
parsing log, completed traces :: 100%|██████████| 31509/31509 [00:57<00:00, 546.80it/s]
/workspaces/ProMean Lib/opal/data/processor.py:83: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['batch'] = (df.groupby('caseID').ngroup() // self.batch_size) + 1


In [13]:
mapper = LogMapper(data=event_data, output_encoding='z3')
z3_data = mapper.get_mapped_data()

Building mappings for logs...


  0%|          | 0/16 [00:00<?, ?it/s]2025-08-11 14:02:34,488 | INFO: Translating YARRRML mapping to [R2]RML
INFO | 2025-08-11 14:02:34,488 | Translating YARRRML mapping to [R2]RML
2025-08-11 14:02:34,490 | INFO: RML content is created!
INFO | 2025-08-11 14:02:34,490 | RML content is created!
2025-08-11 14:02:34,496 | INFO: Mapping has been syntactically validated.
INFO | 2025-08-11 14:02:34,496 | Mapping has been syntactically validated.
2025-08-11 14:02:34,498 | INFO: Translation has finished successfully.
INFO | 2025-08-11 14:02:34,498 | Translation has finished successfully.
2025-08-11 14:02:34,508 | INFO: Translating YARRRML mapping to [R2]RML
INFO | 2025-08-11 14:02:34,508 | Translating YARRRML mapping to [R2]RML
2025-08-11 14:02:34,509 | INFO: RML content is created!
INFO | 2025-08-11 14:02:34,509 | RML content is created!
2025-08-11 14:02:34,516 | INFO: Mapping has been syntactically validated.
INFO | 2025-08-11 14:02:34,516 | Mapping has been syntactically validated.
2025-08-1

Mappings built.
Generating knowledge graph...


INFO | 2025-08-11 14:02:35,287 | 11 mapping rules retrieved.
INFO | 2025-08-11 14:02:35,297 | Mapping partition with 11 groups generated.
INFO | 2025-08-11 14:02:35,299 | Maximum number of rules within mapping group: 1.
INFO | 2025-08-11 14:02:35,299 | Mappings processed in 0.519 seconds.
INFO | 2025-08-11 14:02:36,870 | Number of triples generated in total: 23614.


Knowledge graph generated.
Knowledge graph 1/16 generated.
Generating knowledge graph...


...

Knowledge graph generated.
Knowledge graph 14/16 generated.
Generating knowledge graph...


INFO | 2025-08-11 14:03:11,740 | 11 mapping rules retrieved.
INFO | 2025-08-11 14:03:11,754 | Mapping partition with 11 groups generated.
INFO | 2025-08-11 14:03:11,755 | Maximum number of rules within mapping group: 1.
INFO | 2025-08-11 14:03:11,756 | Mappings processed in 0.540 seconds.
INFO | 2025-08-11 14:03:13,130 | Number of triples generated in total: 22243.


Knowledge graph generated.
Knowledge graph 15/16 generated.
Generating knowledge graph...


INFO | 2025-08-11 14:03:15,046 | 11 mapping rules retrieved.
INFO | 2025-08-11 14:03:15,057 | Mapping partition with 11 groups generated.
INFO | 2025-08-11 14:03:15,058 | Maximum number of rules within mapping group: 1.
INFO | 2025-08-11 14:03:15,059 | Mappings processed in 0.538 seconds.
INFO | 2025-08-11 14:03:16,380 | Number of triples generated in total: 17015.


Knowledge graph generated.
Knowledge graph 16/16 generated.
Z3 literal set generated (1/16)
Z3 literal set generated (2/16)
Z3 literal set generated (3/16)
Z3 literal set generated (4/16)
Z3 literal set generated (5/16)
Z3 literal set generated (6/16)
Z3 literal set generated (7/16)
Z3 literal set generated (8/16)
Z3 literal set generated (9/16)
Z3 literal set generated (10/16)
Z3 literal set generated (11/16)
Z3 literal set generated (12/16)
Z3 literal set generated (13/16)
Z3 literal set generated (14/16)
Z3 literal set generated (15/16)
Z3 literal set generated (16/16)


In [14]:
len(z3_data)

359577

In [ ]:
len(mapper.data.data)

59801

In [16]:
str(z3_data[5008])

'hasResource(E_1174728, R_User_29)'